# ToDo List
1. V Implement params struct, save on train 
2. V Implement dynamic batch loading
3. Normalize/document the dataset format
4. Implement training with warm-start and progressive saving
5. V Implement evaluation on trained models
6. V Separate train/val split on skorch dataset (index list)
7. Include time since admission as feature to predict LoS, else predict time until discharge on hourly (==LoS on admit)
8. V Include sequence lengths, padding, and masking for LSTM model
9. V Handle NaNs
1. V Data flow diagram:  sources, expansions/extractions, dimensions
2. V Time series prediction model:  training process and evaluation
3. V Basic RNN trained
4. V Preliminary results on extant features

In [5]:
# Basic utilities
import importlib
import datetime
import json
import pickle
import math
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm_notebook as tqdm
import logging
import argparse
import os
from pprint import pprint, pformat

# Machine Learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Basis Expansion (external package)
from basis_expansions import (Binner, Polynomial, 
                              LinearSpline, CubicSpline,
                              NaturalCubicSpline)
from dftransformers import ColumnSelector, FeatureUnion, Intercept, MapFeature

# Deep Learning
import torch
print("PyTorch Version:  {}".format(torch.__version__))
from torch import nn
import torch.nn.functional as F
import skorch
from skorch import NeuralNetRegressor
from skorch.callbacks import Callback

# Plotting and visualization
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, Label, Layout
from IPython.display import display, clear_output
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# Local files
import utils 
importlib.reload(utils)

print("Setup complete!")

PyTorch Version:  1.0.1.post2
Setup complete!


# Parameters

In [9]:
params = utils.Params(
    {
        "output_type": "regression",  # Classification vs Regression
        "project_dir": "..",  # Top-level path for this project
        "data_dir": "Data",  # Relative data path
        "cohort_file": "itan_cohort_v.h5",  # File path to the cohort dataset (hdf5)
        "sample_dir": "Data/itan_hourly_encounter_splits",  # Directory to the split patient hourly data
        "label": "LOS",  # Column name of target/label in cohort dataframe
        "cohort_features": ["ADM_LAPS2", "ADM_COPS2", "SEX", "AGE"],  # Which cohort-level features to use
        "hourly_features": ["LAPS2"],#, "IMAR_IM_GROUP"],  # Which patient hourly features to use
        "pad_length": 12,  # maximum number of hours of each time-series to include
        "pad_value": 0,  # Value to pad variable-length input sequences with (unused), and to fill NaNs
        "hourly_embedding_fill_value": 0,  # Value to pad variable-length input sequences with (unused), and to fill NaNs
        "predict_early": 0,  # Number of hours to index LSTM output.  0 to use all available (max=sequence length)
        "association_history": 0,  # How many hours to look back in unit occupancy to define patient associations
        "patient_hourly_hidden_size": 8,  # hidden size of LSTM output
        "unit_hourly_hidden_size1": 8,  # hidden size of LSTM output
        "unit_hourly_hidden_size2": 8,  # hidden size of LSTM output
        "num_hourly_output_features": 8,  # hidden size of LSTM output
        "num_layers": 1,  # number of stacked LSTM layers (each with same hidden_size)
        "model_name": "hierarchical_net",  # which model to train ('linear', 'lstm')
        "num_epochs": 5,  # number of training epochs.  Note:  can rerun the training cell if warm_start=True
        "batch_size": 1,  # number of samples per batch.  Usually limited by GPU memory.
        "learning_rate": 0.01,  # optimizer learning rate
        "dropout_prob": 0.0,  # dropout regularization probability (0 for not implemented)
        "cross_validation_ratios": {"train": 0.8, "val": 0.1, "test": 0.1},  # Fraction of data in each CV split
        "split_file": "set_splits_all_FAC1A.json",  # Where to save the CV splits
        "subset": 0,  # train/evaluate on a smaller subset of the data for testing purposes, reduced runtime
        "subset_facility": "FAC_1_A", # train/evaluate on a subset from a single facility, default None
        "random_seed": 7532941,  # RNG seed for PyTorch (weight initialization and dataset shuffling per-epoch)
        "normalize_cohort": True,  # Whether to normalize the cohort features (mean 0, std 1)
    }
)

# Where to save trained weights, log files, parameter file, results, etc.
model_dir = os.path.join(params.project_dir, 'Code/experiments/', params.model_name)

# Set the logger
utils.set_logger(os.path.join(model_dir, 'ITAN_RNN.log'), logging.INFO) #  DEBUG  INFO

# Set the PyTorch seed for shuffline and weight initialization reproducibility
torch.manual_seed(params.random_seed)

# Save params to file
params.save(os.path.join(model_dir, 'params.json'))
logging.info("Parameters:\n{}".format(pformat(params.__dict__)))
logging.info("Parameters set and saved to file")

Parameters:
{'association_history': 0,
 'batch_size': 1,
 'cohort_features': ['ADM_LAPS2', 'ADM_COPS2', 'SEX', 'AGE'],
 'cohort_file': 'itan_cohort_v.h5',
 'cross_validation_ratios': {'test': 0.1, 'train': 0.8, 'val': 0.1},
 'data_dir': 'Data',
 'dropout_prob': 0.0,
 'hourly_embedding_fill_value': 0,
 'hourly_features': ['LAPS2'],
 'label': 'LOS',
 'learning_rate': 0.01,
 'model_name': 'hierarchical_net',
 'normalize_cohort': True,
 'num_epochs': 5,
 'num_hourly_output_features': 8,
 'num_layers': 1,
 'output_type': 'regression',
 'pad_length': 12,
 'pad_value': 0,
 'patient_hourly_hidden_size': 8,
 'predict_early': 0,
 'project_dir': '..',
 'random_seed': 7532941,
 'sample_dir': 'Data/itan_hourly_encounter_splits',
 'split_file': 'set_splits_all_FAC1A.json',
 'subset': 0,
 'subset_facility': 'FAC_1_A',
 'unit_hourly_hidden_size1': 8,
 'unit_hourly_hidden_size2': 8}
Parameters set and saved to file


# Temporal Cross-Validation within each Facility

In [10]:
# Load sample IDs and set up dataset
cohort_df = pd.read_hdf(os.path.join(params.project_dir, params.data_dir, params.cohort_file),
                                      start=0, 
                                      stop=-1)
cohort_df.columns = [c.split(':')[1] for c in cohort_df.columns] # clean sim/src headers for fuzzed dataset

# Check CV Parameters
assert (sum([params.cross_validation_ratios[split] for split in params.cross_validation_ratios.keys()]) == 1), "Cross validation splits must sum to 1:\n{}".format(params.cross_validation_ratios)

# Split separately for each facility
all_facilities = list(cohort_df["FAC_ID"].unique())
all_facilities.sort()
cohort_df["CV_FAC_SPLIT"] = 'none'
with tqdm(total=len(all_facilities), desc="Facilities") as t:
    for facility in all_facilities:
        logging.debug("Splitting patients for facility {}".format(facility))
        facility_df = cohort_df.loc[cohort_df["FAC_ID"] == facility, :]

        # Split indices for appropriate number/ratio of patients (not days) per split
        N_patients = len(facility_df)
        train_split_index = math.floor(N_patients * params.cross_validation_ratios["train"])
        val_split_index = train_split_index + math.floor(N_patients * params.cross_validation_ratios["val"])

        # Sort by admit date and find boundaries
        facility_df = facility_df.sort_values(["ADM_DATE"]).reset_index(col_fill='cohort_index')
        start_date = facility_df.loc[0, "ADM_DATE"]
        end_date = facility_df.loc[N_patients-1, "ADM_DATE"]
        train_boundary_date = facility_df.loc[train_split_index, "ADM_DATE"]
        val_boundary_date = facility_df.loc[val_split_index, "ADM_DATE"]


        # Assign to cross-validation splits
        cohort_df.loc[((cohort_df["FAC_ID"] == facility) &
                       (cohort_df["DISCH_DATE"] < train_boundary_date)), "CV_FAC_SPLIT"] = 'train'
        cohort_df.loc[((cohort_df["FAC_ID"] == facility) &
                       (cohort_df["ADM_DATE"] >= train_boundary_date) &
                       (cohort_df["DISCH_DATE"] < val_boundary_date)), "CV_FAC_SPLIT"] = 'val'
        cohort_df.loc[((cohort_df["FAC_ID"] == facility) &
                       (cohort_df["ADM_DATE"] >= val_boundary_date)), "CV_FAC_SPLIT"] = 'test'
        t.update()

logging.debug("\nCross-validation split counts:\n{}\n\nCross-validation split ratios:\n{}\n".format(
    cohort_df["CV_FAC_SPLIT"].value_counts(),
    cohort_df["CV_FAC_SPLIT"].value_counts() / len(cohort_df)))


# Restrict to samples with hourly data available
hourly_sample_ids = [np.int64(f.split('.')[0].split('_')[-1])
                     for f in os.listdir(os.path.join(params.project_dir, params.sample_dir))
                     if f.endswith('.h5')]
logging.debug("Loaded {} hourly sample IDs:\n{}".format(len(hourly_sample_ids), hourly_sample_ids[:3]))
cohort_df = cohort_df.loc[cohort_df["ENCOUNTER_ID"].astype(np.int64).isin(hourly_sample_ids), :]
logging.debug("{} samples remaining after dropping samples which don't have hourly data\n".format(len(cohort_df)))

# Subset to a single facility
if params.subset_facility is not None:
    cohort_df = cohort_df.loc[(cohort_df["FAC_ID"] == params.subset_facility), :]
    logging.info("Subsetted to {} samples in facility {} only".format(len(cohort_df), params.subset_facility))

# Split to lists of integers for constructing Skorch datasets
cv_split_samples = {}
for split in ["train", "val", "test"]:
    logging.info("Building {} split sample IDs".format(split))
    
    # Select the matching cohort encount IDs
    samples_df = cohort_df.loc[(cohort_df["CV_FAC_SPLIT"] == split), ["ADM_DATE", "ENCOUNTER_ID"]]
    
    # Sort chronologically to maximize concurrency on subset
    samples_df.sort_values(by="ADM_DATE", ascending=True, inplace=True)
    logging.info("Sorted samples:\n{}".format(samples_df[:10]))
    
    # Format as integers for size/index speed
    cv_split_samples[split] = list(samples_df["ENCOUNTER_ID"].astype(np.int64).reset_index(drop=True))
    
    # Select a smaller subset of each split for quick code testing
    if params.subset > 0:
        cv_split_samples[split] = cv_split_samples[split][:params.subset]
    logging.info("{} split:  {} samples".format(split, len(cv_split_samples[split])))


# Save CV splits to file (for easier reload if using subsets, alternate datasets on different machines, etc.)
split_filepath = os.path.join(params.project_dir, "Code/", "experiments/", params.model_name, params.split_file)
logging.info("Saving set splits to {}".format(split_filepath))
with open(split_filepath, 'w') as f:
    json.dump(cv_split_samples, f, indent=4)

Subsetted to 47384 samples in facility FAC_1_A only
Building train split sample IDs
Sorted samples:
                            ADM_DATE  ENCOUNTER_ID
31399   Fri Apr 01 00:00:00 GMT 2016  330000225236
58460   Fri Apr 01 00:00:00 GMT 2016  330000774040
41173   Fri Apr 01 00:00:00 GMT 2016  330000811143
87780   Fri Apr 01 00:00:00 GMT 2016  330000103855
197102  Fri Apr 01 00:00:00 GMT 2016  330000253566
162096  Fri Apr 01 00:00:00 GMT 2016  330000016784
201044  Fri Apr 01 00:00:00 GMT 2016  330000522750
86013   Fri Apr 01 00:00:00 GMT 2016  330000986695
85769   Fri Apr 01 00:00:00 GMT 2016  330000777166
257755  Fri Apr 01 00:00:00 GMT 2016  330000678481
train split:  31009 samples
Building val split sample IDs
Sorted samples:
                            ADM_DATE  ENCOUNTER_ID
234716  Tue May 24 00:00:00 GMT 2016  330000812759
47862   Tue May 24 00:00:00 GMT 2016  330000639774
47532   Tue May 24 00:00:00 GMT 2016  330000773481
259183  Tue May 24 00:00:00 GMT 2016  330000200713
94882   Tu

# Batch Data Loader

In [ ]:
logging.debug("Defining dataset class")
class ITANDataset(skorch.dataset.Dataset):
    def __init__(self, sample_ids, params):
        """ Construct ITAN Dataset for dynamic data loading with Skorch
        Args:
            sample_ids: (pandas.Series or list)  Sample (encounter) ids for indexing cohort data and hourly files
        """
        self.params = params
        self.sample_ids = sample_ids
        logging.debug("Sample IDs:  {}\n{}".format(self.sample_ids.shape, self.sample_ids[:5]))
        
        # Keep all cohort data in memory
        if params.cohort_features:
            self.cohort_df = self.load_cohort_features(params)
            self.cohort_features = [c for c in self.cohort_df.columns if c != params.label]
            
            # Drop samples that don't have cohort data
            self.sample_ids = self.cohort_df.index
            
        # No hourly feature expansion implemented
        self.hourly_features = params.hourly_features
            
        # Split features and label/target
        self.Y = self.cohort_df.loc[:, [params.label]].values.astype(np.float32)
        self.cohort_df.drop([params.label], axis=1, inplace=True)
        
        if params.normalize_cohort:
            self.cohort_df = self.normalize(self.cohort_df)
        
        logging.debug("Final labels:  {}\n{}".format(self.Y.shape, self.Y[:3]))
        logging.debug("Final cohort features:  {}\n{}".format(self.cohort_df.shape, self.cohort_df[:3]))

    def normalize(self, df):
        """ Normalize all columns in a dataframe (cohort_df) to mean 0 and std 1
        """
        logging.debug("Normalizing features")
        scaler = StandardScaler()
        x = df.values #returns a numpy array
        x_scaled = scaler.fit_transform(x)
        df[:] = x_scaled
        return df
            
    def load_cohort_features(self, params):
        """ Load all cohort-level features from file and preprocess
        Args:
            params:  dictionary of parameters
        Return:
            cohort_df:  (pandas.DataFrame)  All cohort-level data indexed by encounter ID
        """
        logging.debug("Loading cohort features")
        cohort_df = pd.read_hdf(os.path.join(params.project_dir, params.data_dir, params.cohort_file),
                                start=0, 
                                stop=-1) # note: loading all features and dropping after cleaning sim/src
        cohort_df.columns = [c.split(':')[1] for c in cohort_df.columns] # clean sim/src headers for fuzzed dataset
        cohort_df.set_index("ENCOUNTER_ID", inplace=True)
        cohort_df = cohort_df.loc[:, [params.label]+params.cohort_features]
        logging.debug("Loaded {} samples from cohort-level data".format(cohort_df.shape))
        
        # Drop rows with duplicate Encounter IDs
        cohort_df = cohort_df[~cohort_df.index.duplicated(keep='first')]
        logging.debug("{} samples after dropping duplicate encounter IDs".format(cohort_df.shape))
        
        # Select out the rows for encounter IDs in the current CV split
        cohort_df = cohort_df.loc[self.sample_ids, :]
        logging.debug("Loaded cohort data:  {}\n{}".format(cohort_df.shape, cohort_df[:5]))
        logging.debug("Cohort datatypes:\n{}".format(cohort_df.dtypes))
        
        # Drop rows with NaN
        cohort_df.dropna(axis='index', how='any', inplace=True)
        logging.debug("{} samples after dropping rows with missing values".format(len(cohort_df)))
        
        # Expand categorical features with dummy indicators
        categorical_features = [c for ci, c in enumerate(cohort_df.columns) if cohort_df.dtypes[ci]==object]
        if categorical_features:
            logging.debug("Expanding categorical variables to dummy indicators:  {}".format(categorical_features))
            cohort_df = pd.get_dummies(cohort_df, columns=categorical_features).astype(np.float32)
            logging.debug("Expanded/binarized data:\n{}".format(cohort_df[:5]))
        else:
            logging.debug("No categorical variables to expand")
        
        # Return the expanded cohort data, with labels (targets, Y) included
        return cohort_df
    
    def __len__(self):
        return len(self.sample_ids)
    
    def __getitem__(self, index):
#         logging.debug("getitem index: {}".format(index))
        if self.hourly_features:
            filepath = os.path.join(self.params.project_dir,
                                    self.params.sample_dir,
                                    "itan_hourly_enc_{}.h5".format(self.sample_ids[index]))

    #         logging.debug("Sample cohort data:  {},   {}".format(sample_cohort_df.shape, sample_cohort_df))

            # Dynamically load patient hourly data
            sample_hourly_df = pd.read_hdf(filepath, key="hourly").reset_index()
            sample_hourly_df.drop('index', axis=1, inplace=True)
            sample_hourly_df.columns = [c.split(':')[1] for c in sample_hourly_df.columns] # clean sim/src headers for fuzzed dataset
#         logging.debug("Sample hourly data:  {},   {}".format(sample_hourly_df.shape, sample_hourly_df.columns))
            sample_pad = min(params.pad_length, len(sample_hourly_df))
        else:
            # No hourly features, don't tile cohort features
            sample_pad = 1

        # Construct feature array:  sklearn/skorch LSTM format is batch-first:  (batch, sequence, feature)
        # Pad all sequences to fixed length
        Xi = np.full(shape=(params.pad_length, len(self.hourly_features)+len(self.cohort_features)),
                     fill_value=params.pad_value,
                     dtype=np.float32)
#         logging.debug("Xi init: {}".format(Xi.shape))
        
        # Fill with hourly data
        if self.hourly_features:
            sample_hourly_df.fillna(params.pad_value, inplace=True)
            Xi[:(sample_pad), :len(self.hourly_features)] = sample_hourly_df.loc[:(sample_pad-1), self.hourly_features].values
        
        # Fill with cohort data tiled on sequency (temporal) dimension
        if self.cohort_features:
            sample_cohort_df = self.cohort_df.loc[[self.sample_ids[index]], :].iloc[[0], :]
#             logging.debug("sample cohort df: {}\n{}".format(sample_cohort_df.shape, sample_cohort_df))
#             logging.debug("tiled cohort df:  {}".format(np.tile(sample_cohort_df.values, (sample_pad, 1)).shape))
            Xi[:(sample_pad), len(self.hourly_features):] = np.tile(sample_cohort_df.values, (sample_pad, 1))
        
        # Optionally reset the sequence length to early-output
        # Most likely used in evaluation after training on full length
        # Leave at 0 to make prediction at end of sequence
        if self.params.predict_early > 0:
            output_index = min(self.params.predict_early, sample_pad)
        else:
            output_index = sample_pad
            
        yi = self.Y[index]
        return (Xi, output_index), yi

    
# Construct Skorch datasets for training
logging.info("Constructing datasets for each cross-validation split.")
datasets = {}
for split in ["train", "val"]:
    datasets[split] = ITANDataset(cv_split_samples[split], params)
    logging.info("{} split:  {} samples:\n{}\nX: {}\nY: {}".format(split, len(cv_split_samples[split]), cv_split_samples[split][:3], None, datasets[split].Y.shape))
    
    logging.info("y values:  {}:  mean {:.2f} +/- {:.2f}".format(
        datasets[split].Y.shape,
        np.mean(datasets[split].Y),
        np.std(datasets[split].Y)))  
    
    # Test iteration through dataset
    if False:
        with tqdm(total=len(datasets[split]), desc="Testing dataset iterator") as st:
            for sample in datasets[split]:
                st.update()
logging.info("\nDatasets defined!")

# Model Definition

In [59]:
# Neural Network definition
class LinearRegressorModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super().__init__()

        # Define network layers
        self.denseX = nn.Linear(len(params.hourly_features)+len(params.cohort_features)+1, 1) #*params.pad_length
        
        # Manual initialization
        self.denseX.weight.data.fill_(0.0)
        self.denseX.bias.data.fill_(1.0)

    def forward(self, X, **kwargs):
        shape_debug = False
        if shape_debug: logging.info("input: {}\n{}".format(X.size(), X.dtype))
        X = torch.mean(X, dim=1, keepdim=False)
        if shape_debug: logging.info("reshape: {}".format(X.size()))
        X = self.denseX(X)
        if shape_debug: logging.info("output: {}".format(X.size()))
        return X
    

# Recurrent Neural Network definition
class LSTMRegressorModule(nn.Module):
    def __init__(self, params):
        super().__init__()
        input_size = len(params.hourly_features)+len(params.cohort_features)+1
        hidden_size = params.hidden_size
        
        # Define network layers
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.output_regression = nn.Linear(hidden_size, 1)
        
#         self.bn1 = 

        
    def forward(self, inputs):
        shape_debug = False
        sequences, lengths = inputs
        if shape_debug: logging.info("input:  {}".format(sequences.size()))
        if shape_debug: logging.info("lengths:  {}".format(lengths.size()))
        
        output, (h_n, c_n) = self.lstm1(sequences)
        X = output
        if shape_debug: logging.info("LSTM 1: {}".format(X.size()))
            
        length_indices = lengths.view(-1, 1, 1).expand(X.size(0), 1, X.size(2)).long() - 1
        if shape_debug: logging.info("Slice Indices:  {}".format(length_indices.size()))
#         if shape_debug: logging.info("Slice Indices:  {}\n{}".format(length_indices.size(), length_indices))
        
#         X = torch.index_select(output, dim=1, index=lengths)
        X = X.gather(1, length_indices)
        if shape_debug: logging.info("Length-indexed outputs:  {}".format(X.size()))
            
        X = X.squeeze(1)
        if shape_debug: logging.info("Last Output in Sequence:  {}\n{}".format(X.size(), X.dtype))
        
        X = self.output_regression(X)
        if shape_debug: logging.info("Output: {}".format(X.size()))
        return X
    
    
class ITANStrainNetworkModule(nn.Module):
    def __init__(self, params):
        """ Load parameters and initialize NN model
        
        Params:
            output_type:  (string) target type, determines output layer
            include_strain_variables:  (bool) whether to use the unit and network-level strain connections
            hourly_features:  (list) list of hourly feature column names
            cohort_features:  (list) list of cohort-level (on admission) feature column names
            patient_hourly_hidden_size:  (int) hidden size of LSTM for all batch-associated patient hourly features
            unit_hourly_hidden_size:  (int) hidden size of set accumulator for embedding all patients simultaneously occupying a unit
            num_hourly_output_features:  (int)
        """
        super().__init__()
        # Compute and save parameters
        self.params = params
        self.input_size = len(params.hourly_features)+len(params.cohort_features)+1
        self.num_output_features = params.num_hourly_output_features+len(params.cohort_features)
        
        # Define internal network layers and RNN cells
        self.lstm_associated_patient = nn.LSTM(input_size=self.input_size,
                                               hidden_size=self.params.patient_hourly_hidden_size,
                                               num_layers=1,
                                               batch_first=True,
                                               bidirectional=False)
        
        self.patient_set_accumulate = nn.LSTM(input_size=self.params.patient_hourly_hidden_size,
                                              hidden_size=self.params.unit_hourly_hidden_size1,
                                              num_layers=1,
                                              batch_first=True,
                                              dropout=params.dropout_prob,
                                              bidirectional=False)#True)
        
        self.lstm_unit = nn.LSTM(input_size=self.params.unit_hourly_hidden_size1,
                                 hidden_size=self.params.unit_hourly_hidden_size2,
                                 num_layers=1,
                                 batch_first=True,
                                 bidirectional=False)
        
        self.lstm_sample_patient = nn.LSTM(input_size=self.params.patient_hourly_hidden_size+len(self.params.cohort_features),
                                           hidden_size=self.params.num_hourly_output_features,
                                           num_layers=1,
                                           batch_first=True,
                                           bidirectional=False)
        
        # Define output layer
        if params.output_type == "classification":
            self.output_layer = nn.Softmax(self.num_output_features, 2)
        elif params.output_type == "regression":
            self.output_layer = nn.Linear(self.num_output_features, 1)
        else:
            logging.warning("Unknown output type:  {}".format(self.output_layer))
            
        
    def forward(self, inputs):
        """ Inputs:  tuple built by data_loader
        
        Def:  Batch patients/samples:  Exactly batch_size randomly selected set of patients to make an output prediction for the current batch.  
        Def:  Patients associated with batch:  Unknown number of patients who were in any unit that one of the batch patients eventually occupied
              This allows for building a unit history prior to the batch patients' occupancy
              Always at least batch_size, additional patients may be clipped to a randomly selected subset for size restrictions and regularization
        Def:  pad_length:  maximum number of hours of data for any patient in the batch
        Def:  time_max:  Total number of hours from first sample of any batch-associated patient to last sample of any
        
        Args:
            inputs:  (tuple)  all batch data generated per-sample by data loader and batched by Skorch
            batch_samples:  (batch_patients)  boolean mask for which patients are being predicted out of all patients associated with the batch
                            sums to batch_size
            patient_inputs:  (tuple)  all data for patients associated with the batch
            
            patient_static_features:  (batch_patients x num_patient_static_features)  static features for all patients associated with the batch
            Xpatient_locations:  (batch_patients x time_max x num_units)  location of each patient associated with the batch at each time (global time)
            unit_hourly_occupants:  (batch_units x time_max x max_occupancy)  indices into batch patients for all occupants of each unit at each global time
            patient_hourly_features:  (batch_patients x pad_length x num_patient_hourly_features)  full history of patient hourly features for each patient associated with batch
            patient_hourly_lengths:  (batch_patients)  number of valid samples in hourly sequences (clipped to pad_length)
            patient_hourly_timemask:  (batch_patients x time_max)  boolean mask to map personal time indices to global time index
        """
        shape_debug = True
        # Extract bundled batch data
        patient_static_features, patient_hourly_locations, patient_hourly_features, patient_hourly_lengths = inputs
        
        if shape_debug: logging.info("Patient Static Features: {}".format(patient_static_features.size()))
        if shape_debug: logging.info("Patient Hourly Locations: {}".format(patient_hourly_locations.size()))
        if shape_debug: logging.info("Patient Hourly Features: {}".format(patient_hourly_features.size()))
        if shape_debug: logging.info("Patient Hourly Lengths: {}".format(patient_hourly_lengths.size()))
        
        # Wrap up time-history for each patient associated with batch
        # Keep full RNN history (1-directional)
        patient_personal_hourly_embeddings, _ = self.lstm_associated_patient(patient_hourly_features)  # (batch_patients x pad_length x patient_hourly_hidden_size)
        if shape_debug: logging.info("Patient Hourly Embeddings personal-time: {}".format(patient_personal_hourly_embeddings.size()))

        # Get batch-associated patient hourly embeddings on global time index, index out patients for each unit at each time
        # The shape of mask must be broadcastable with the shape of the underlying tensor.
        patient_global_hourly_embeddings = torch.new_full(size=(self.params.time_max, self.params.patient_hourly_hidden_size1), fill_value=self.params.hourly_embedding_fill_value, dtype=bool)
        patient_global_hourly_embeddings.masked_scatter_(patient_hourly_timemask, patient_personal_hourly_embeddings)  # (batch_patients x time_max x patient_hourly_hidden_size)
        if shape_debug: logging.info("Patient Hourly Embeddings global-time: {}".format(patient_global_hourly_embeddings.size()))
            
        # Group batch-associated patient hourly embeddings by simultaneous unit occupancy and expand dimension appropriately
        # gather:  index shape is same as output shape (but broadcast???)
        # index provided by data loader
        unit_patient_hourly_embeddings = torch.gather(patient_hourly_embeddings, dim=0)#, unit_hourly_occupants)  # (batch_units x time_max x max_occupancy x patient_hourly_hidden_size)
        if shape_debug: logging.info("Unit-Patient Hourly Embeddings reshaping: {}".format(unit_patient_hourly_embeddings.size()))
    
        # Merge hourly embeddings over all batch-associated patients in a unit simultaneously
        # Can be done with reduce-max, RNN, shuffle/RNN, or search literature for unordered set combination methods
        # note:  torch.matmul multiplies last 2 dimensions, all preceding are considered as batch
        # note:  may need to re-.view() so batch_size and max_time are both in batch dimension, and reshape after
        unit_patient_hourly_embeddings = unit_patient_hourly_embeddings.view(-1, max_occupancy, patient_hourly_hidden_size)  # (batch_units*time_max x max_occupancy x patient_hourly_hidden_size)
        output, (h_n, c_n) = self.patient_set_accumulate()  
        occupancy_indices = unit_hourly_occupants.view(-1, 1, 1).expand(output.size(0), 1, output.size(2)).long() - 1
        unit_hourly_embeddings = output.gather(1, occupancy_indices) # (batch_units*time_max x unit_hourly_hidden_size)
        unit_hourly_embeddings = unit_hourly_embeddings.view(batch_units, time_max, unit_hourly_hidden_size)  # (batch_units x time_max x unit_hourly_hidden_size)
        if shape_debug: logging.info("Unit Hourly Embeddings patient-wrap: {}".format(unit_hourly_embeddings.size()))
            
        # Wrap up time-history for each unit associated with batch
        # Unit summaries change as patients move in and out
        # Keep full history
        unit_accumulated_embeddings, _ = self.lstm_unit(unit_hourly_embeddings)  # (batch_units, time_max, unit_hourly_hidden_size2)
        if shape_debug: logging.info("Unit Hourly Embeddings time-wrap: {}".format(unit_accumulated_embeddings.size()))
        
        # Select out the unit-level data for each batch patient at each global time
        # Duplicate data as necessary
        patient_global_hourly_unit_features = torch.gather(unit_accumulated_embeddings)#, dim=0, patient_locations)  # (batch_samples x pad_length x unit_hourly_hidden_size)
        if shape_debug: logging.info("Patient Unit Hourly Embeddings reshaping/duplication in global-time: {}".format(patient_global_hourly_unit_features.size()))
        
        # Re-index to personal (sample) time for each batch patient
        # At each personal time step in patient_sequences, get patient location, index the accumulated unit data, and concat to row for patient/time
        patient_personal_hourly_unit_features = torch.gather(patient_global_hourly_unit_features, dim=1)#, patient_hourly_timemask)
        if shape_debug: logging.info("Patient Unit Hourly Embeddings in personal-time: {}".format(patient_personal_hourly_unit_features.size()))
            
        # Merge patient hourly and the unit data for their current location
        patient_hourly_features = concat(patient_sequences, patient_hourly_unit_features)  # (batch_samples x pad_length x unit_hourly_hidden_size+num_hourly_features)
        if shape_debug: logging.info("Patient Hourly Features (unit & personal): {}".format(patient_hourly_features.size()))
        
        # Wrap up time-history for each patient as they move through units
        patient_accumulated_features = self.lstm_sample_patient(patient_data_inst, patient_lengths)  # (batch_samples x pad_length x patient_hourly_hidden_size2)
        if shape_debug: logging.info("Patient Hourly Embeddings (wrap all features over personal time): {}".format(patient_accumulated_features.size()))
            
        # Tile and append patient static features
        patient_all_features = concat(patient_accumulated_features, torch.tile(patient_static_features))  # (batch_samples x pad_length x patient_hourly_hidden_size2+num_static_features)
        if shape_debug: logging.info("All Patient Features (hourly embeddings and static features): {}".format(patient_all_features.size()))
            
        # Make a prediction (regression/classification) for each patient in batch (at each time-elapsedLoS?)
        patient_hourly_pred = self.output_layer(patient_data_accumulate)
        if shape_debug: logging.info("Patient Predictions at each personal-timestep: {}".format(patient_hourly_pred.size()))
        return patient_hourly_pred
    
    
# Select a model to train
if params.model_name == "linear":
    nn_module = LinearRegressorModule
elif params.model_name == "lstm":
    nn_module = LSTMRegressorModule
elif params.model_name == "hierarchical_net":
    nn_module = HierarchicalNetworkModule
else:
    print("Unknown model {}, cannot construct".format(params.model))
    raise NotImplementedError

    
# # Evaluation Metrics
# class R_squared(Callback):
#     def on_epoch_end(dataset_train, dataset_val):
#         y_train_pred = self.predict(dataset_train)
#         y_val_pred = self.predict(dataset_val)
        
#         r2_train = r2_score(dataset_train.Y, y_train_pred)
#         r2_val = r2_score(dataset_val.Y, y_val_pred)

# callbacks = [
#     ("R-squared", R_squared())
# ]

callbacks = [
    skorch.callbacks.ProgressBar()
]


# NN Regression setup.  Construct model with preset parameters
net = NeuralNetRegressor(
    nn_module(params),
    max_epochs=params.num_epochs,
    lr=params.learning_rate,
    batch_size=params.batch_size,
    callbacks=callbacks,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    verbose=1,
    warm_start=True,
)

# Use sklearn pipeline for preprocessing, cross-validation, etc.
# Any preprocessing must be compatible with Skorch dataset and 3d input (batched multifeature timeseries)
# pipe = Pipeline([
#         ('scale', StandardScaler()),  # note: doesnt work with multidimensional input
#         ('nnet', net)])
# model = pipe
model = net
logging.info("Model defined!")

Model defined!


# Model Training
Note: can rerun this cell with warm-start to continue training

In [64]:
# Load parameters from file
# params.model_name = 'test'  # optionally override the model to train
# params = utils.Params(os.path.join(params.project_dir, 'Code/experiments/', params.model_name, 'params.json'))
params.update(os.path.join(params.project_dir, 'Code/experiments/', params.model_name, 'params.json'))


# Reload weights if necessary
# hot_start = False
# if hot_start:
#     logging.info("Hot-starting model with trained weights from {}".format(model_dir))
#     with open(os.path.join(model_dir, 'trained_weights.pkl'), 'rb') as f:
#         model = pickle.load(f)
# else:
#     logging.info("Re-initializing model from scratch")

# Use CUDA if available


# Fit the model
logging.info("Fitting model...")
model.fit(datasets["train"], y=None)
logging.info("Fit complete!")

# Save trained weights
# with open(os.path.join(model_dir, 'trained_weights.pkl'), 'wb') as f:
#     pickle.dump(model, f)
    


    
# Log results



Fitting model...


TypeError: batch must contain tensors, numbers, dicts or lists; found object

# Evaluate fitted model on training and validation sets

In [ ]:
print("Evaluating fitted model on training and validation sets")
print("y_train:       {}:  mean {:.2f} +/- {:.2f}".format(
    datasets["train"].Y.shape,
    np.mean(datasets["train"].Y),
    np.std(datasets["train"].Y)))    
y_train_pred = model.predict(datasets["train"])
print("y_train_pred:  {}:  mean {:.2f} +/- {:.2f}".format(
    y_train_pred.shape,
    np.mean(y_train_pred),
    np.std(y_train_pred)))
r2_train = r2_score(datasets["train"].Y, y_train_pred)
print("R-squared (train):  {:.4f}".format(r2_train))

y_val_pred = model.predict(datasets["val"])
r2_val = r2_score(datasets["val"].Y, y_val_pred)
print("R-squared (val):    {:.4f}".format(r2_val))

# Evaluate on Variable-Length Inputs

In [ ]:
# Initialize arrays for plotting
hour_range = np.arange(1, params.pad_length+1)
logging.info("Evaluating trained model with {} to {} hours of input data".format(hour_range[0], hour_range[-1]))
r2_train = np.zeros(params.pad_length, dtype=np.float32)
r2_val = np.zeros(params.pad_length, dtype=np.float32)

# Run inference on pretrained model with each early-predict time
with tqdm(total=params.pad_length, desc="Early Predict Hours") as lt:
    for li, input_length in enumerate(hour_range):
        # Set parameters for early evaluation
        datasets["train"].params.predict_early = input_length
        datasets["val"].params.predict_early = input_length
        
        # Run new inferences on rebuilt datasets using pretrained model
        logging.debug("Running inferences with pretrained models")
        y_train_pred = model.predict(datasets["train"])
        r2_train[li] = r2_score(datasets["train"].Y, y_train_pred)
        y_val_pred = model.predict(datasets["val"])
        r2_val[li] = r2_score(datasets["val"].Y, y_val_pred)
        lt.update()
        logging.info("R-squared at {:<3} hours:\ttrain = {:.4f}, val = {:.4f}".format(input_length, r2_train[li], r2_val[li]))

# Cleanup parameters
datasets["train"].params.hourly_features = params.hourly_features
datasets["train"].params.predict_early = params.predict_early
datasets["val"].params.hourly_features = params.hourly_features
datasets["val"].params.predict_early = params.predict_early

# Plot Results

In [ ]:
plot_stop = 6

logging.info("R-squared at {:<3} hours:\ttrain = {:.4f}, val = {:.4f}".format(input_length, r2_train[li], r2_val[li]))
# Plot results for evaluation with variable inputs
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5), squeeze=False)
axes[0, 0].plot(hour_range[:plot_stop], r2_train[:len(hour_range)][:plot_stop], color='red', label='train')
axes[0, 0].plot(hour_range[:plot_stop], r2_val[:len(hour_range)][:plot_stop], color='blue', label='val')
axes[0, 0].set_ylabel("R-squared")
axes[0, 0].set_title("Model Evaluation with Incoming Data")
axes[0, 0].set_xlabel("Number of Input Hours")
axes[0, 0].legend(loc='upper left')
axes[0, 0].grid(True)
fig.savefig("RNN_evaluation_over_time_alldata.png", tight_layout=True)

# New Batch Loader for Strain Network

Notes:
1.  Batches are always drawn from same facility
2.  Try to draw batches in close temporal proximity
3.  Perform mini-epochs for each facility?  Could have cyclical training issues, but much more efficient memory-wise.
4.  Hold batch-associated info in memory?


High-Memory Method:
1.  1 facility at a time
2.  Hold all data in CPU memory
3.  Move all hourly data into sparse, 3d array on global timescale (num_patients x time_max x num_features)
4.  Move only necessary hourly data into GPU memory on batching?  Need to control batching indices if so to get all batch-associated patients.

Default solution:
To override batching, need to control torch.utils.data.DataLoader() for train_iterator and valid_iterator in NN
Pass collate_fn() to DataLoader():  merges a list of samples to form a mini-batch.

Alternative solution:
Keep everything in memory, don't have any strain vars in batch, just the sampling indices.  
Then use the data on forward(), instead of __getitem__()
so the data must be available to the network module init, rather than just the dataloader init,
unless a handler/pointer is passed rather than the data

Easiest Method:
Duplicate all necessary strain data per-sample within batch, keep batch size 1 or small


In [70]:
class ITANStrainDataset(skorch.dataset.Dataset):
    def __init__(self, sample_ids, params):
        """ Construct ITAN Dataset for dynamic data loading with Skorch
        Args:
            sample_ids: (pandas.Series or list)  Sample (encounter) ids for indexing cohort data and hourly files
        """
        self.params = params
        self.sample_ids = sample_ids
        logging.debug("Sample IDs:  {}\n{}".format(self.sample_ids.shape, self.sample_ids[:5]))
        
        # Keep all cohort data in memory
        if params.cohort_features:
            self.cohort_df = self.load_cohort_features()
            self.cohort_features = [c for c in self.cohort_df.columns if c != params.label]
            
            # Drop samples that don't have cohort data
            self.sample_ids = self.cohort_df.index
            
        # Split features and label/target
        self.Y = self.cohort_df.loc[:, [params.label]].values.astype(np.float32)
        self.cohort_df.drop([params.label], axis=1, inplace=True)
        
        if params.normalize_cohort:
            self.cohort_df = self.normalize(self.cohort_df)
            
        # Keep all hourly data in memory
        if params.hourly_features:
            self.hourly_features = params.hourly_features
            self.hourly_dataframes, self.global_time_indices = self.load_hourly_features(self.sample_ids)
            
        # Prebuild all patient locations
        self.hourly_patient_locations = self.prebuild_patient_locations(self.sample_ids, self.hourly_dataframes, self.global_time_indices)
        
        # Prebuild all patient associations to index at batch-time
        self.patient_associations = self.prebuild_patient_associations(self.sample_ids, self.hourly_patient_locations, history_hours=params.association_history)

        
    def normalize(self, df):
        """ Normalize all columns in a dataframe (cohort_df) to mean 0 and std 1
        """
        logging.debug("Normalizing features")
        scaler = StandardScaler()
        x = df.values #returns a numpy array
        x_scaled = scaler.fit_transform(x)
        df[:] = x_scaled
        return df
            
    def load_cohort_features(self):
        """ Load all cohort-level features from file and preprocess
        Args:
            params:  dictionary of parameters
        Return:
            cohort_df:  (pandas.DataFrame)  All cohort-level data indexed by encounter ID
        """
        logging.debug("Loading cohort features")
        cohort_df = pd.read_hdf(os.path.join(self.params.project_dir, self.params.data_dir, self.params.cohort_file),
                                start=0, 
                                stop=-1) # note: loading all features and dropping after cleaning sim/src
        cohort_df.columns = [c.split(':')[1] for c in cohort_df.columns] # clean sim/src headers for fuzzed dataset
        cohort_df.set_index("ENCOUNTER_ID", inplace=True)
        cohort_df = cohort_df.loc[:, [self.params.label]+self.params.cohort_features]
        logging.debug("Loaded {} samples from cohort-level data".format(cohort_df.shape))
        
        # Drop rows with duplicate Encounter IDs
        cohort_df = cohort_df[~cohort_df.index.duplicated(keep='first')]
        logging.debug("{} samples after dropping duplicate encounter IDs".format(cohort_df.shape))
        
        # Select out the rows for encounter IDs in the current CV split
        cohort_df = cohort_df.loc[self.sample_ids, :]
        logging.debug("Loaded cohort data:  {}\n{}".format(cohort_df.shape, cohort_df[:5]))
        logging.debug("Cohort datatypes:\n{}".format(cohort_df.dtypes))
        
        # Drop rows with NaN
        cohort_df.dropna(axis='index', how='any', inplace=True)
        logging.debug("{} samples after dropping rows with missing values".format(len(cohort_df)))
        
        # Expand categorical features with dummy indicators
        categorical_features = [c for ci, c in enumerate(cohort_df.columns) if cohort_df.dtypes[ci]==object]
        if categorical_features:
            logging.debug("Expanding categorical variables to dummy indicators:  {}".format(categorical_features))
            cohort_df = pd.get_dummies(cohort_df, columns=categorical_features).astype(np.float32)
            logging.debug("Expanded/binarized data:\n{}".format(cohort_df[:5]))
        else:
            logging.debug("No categorical variables to expand")
        
        # Return the expanded cohort data, with labels (targets, Y) included
        return cohort_df
    
    def load_hourly_features(self, sample_ids, location_column="LOCATION", index_column="LAPS2_TS"):
        """ Load hourly data for all patients listed in self.sample_ids
        
        Params:
            sample_ids: (list)  Which sample IDs to load data for
        Returns:
            hourly_dataframes: (dictionary of DataFrame, keyed by sample_id)
            global_time_indices:  pd.DateTimeIndex?  hourly indices over all give sample IDs
        """
        hourly_dataframes = {}
        
        for sample_id in sample_ids:
            filepath = os.path.join(self.params.project_dir,
                                    self.params.sample_dir,
                                    "itan_hourly_enc_{}.h5".format(sample_id))

            # Load patient hourly data
            sample_hourly_df = pd.read_hdf(filepath, key="hourly").reset_index()
            sample_hourly_df.drop('index', axis=1, inplace=True)
            sample_hourly_df.columns = [c.split(':')[1] for c in sample_hourly_df.columns] # clean sim/src headers for fuzzed dataset

            # Set index to start of each hour (rounded to :00) and drop unused hourly features
            sample_hourly_df[index_column] =  pd.to_datetime(sample_hourly_df[index_column]).dt.round('1H')  
            sample_hourly_df.set_index(index_column, inplace=True)
            sample_hourly_df = sample_hourly_df.loc[:, [location_column]+self.params.hourly_features]
            logging.debug("Sample {} hourly dataframe:\n{}".format(sample_id, sample_hourly_df[:5]))
            
            sample_pad = min(params.pad_length, len(sample_hourly_df))
            
            # Retain hourly data in CPU memory
            hourly_dataframes[sample_id] = sample_hourly_df
            
        # Check time frames and build global time index
        start_stop_times = pd.DataFrame(data=[[sample_hourly_df.index.min(), sample_hourly_df.index.max()]
                                              for sample_id, sample_hourly_df in hourly_dataframes.items()],
                                        columns=["Start", "Stop"])
        
        logging.debug("Start/stop times for each sample ID:\n{}".format(start_stop_times))
        global_time_indices = pd.date_range(start=start_stop_times["Start"].min(),
                                            end=start_stop_times["Stop"].max(),
                                            freq="1H")
        logging.debug("Global time indices over {} hours:\n{}".format(len(global_time_indices), global_time_indices[:5]))
        
        return hourly_dataframes, global_time_indices
        
    def prebuild_patient_locations(self, sample_ids, hourly_dataframes, global_time_indices):
        """ Build matrix of patient locations at each time
            Note:  assumes single facility (FAC_ID not used)
            
        Returns:
            unique_locations: (list) string values of all unique unit locations
            hourly_patient_locations: (pd.DataFrame)  shape=(time_max x num_patients)
        """
        logging.debug("Pre-building all patient locations in global time index")
        # Build list of all possible locations.  Location matrix values will be indices into this list
#         locations = set([])
#         for sample_id in sample_ids:
#             locations += hourly_dataframes[sample_id]["LOCATION", :].unique()
#         locations = list(locations)
        
        hourly_patient_locations = pd.DataFrame(columns=sample_ids, index=global_time_indices)
        with tqdm(total=len(sample_ids), desc="Sample IDs") as ts:
            for sample_id in sample_ids:
                sample_times = hourly_dataframes[sample_id].index
                hourly_patient_locations.loc[sample_times, sample_id] = hourly_dataframes[sample_id]["LOCATION"]
                ts.update()
            
        logging.debug("Built patient locations: {}:\n{}".format(hourly_patient_locations.shape, hourly_patient_locations[:3]))
        return hourly_patient_locations
    
    def prebuild_patient_associations(self, sample_ids, hourly_patient_locations, history_hours=0):
        """ Prebuild array for which patients are associated with which.
            Based on simultaneous location in same facility/unit, or prior occupation within a fixed time window
        
        Params:
            sample_ids:  (list)  which patients for which to find associated other patients (e.g. the batch samples)
            hourly_patient_locations:  (DataFrame)  the patient locations for all samples
            history_hours:  (int) number of hours to lookback when deciding if patients are associated
                            default 0 to require simultaneous occupancy of same unit
        
        Returns:
            associations:  (DataFrame of bool)  indicators for which patients are associated with the given sample patients
        """
        logging.debug("Pre-building all patient associations")
        all_sample_ids = hourly_patient_locations.columns
        associations = pd.DataFrame(0, dtype=np.bool, index=sample_ids, columns=all_sample_ids)
        
        with tqdm(total=len(sample_ids), desc="Sample IDs") as ts:
            for sample_id in sample_ids:
                # Create location history matching array
                # Note:  len(sample_ids) <= len(all_sample_ids)

                # Check which patients were in a matching location at some point in the history limit


                # TODO:  Haven't implemented historical occupation yet
                if (history_hours != 0): raise NotImplementedError

                all_locations = hourly_patient_locations.values
                logging.debug("All locations:  {}\n{}".format(hourly_patient_locations.shape, hourly_patient_locations[:3]))
                sample_locations = np.expand_dims(hourly_patient_locations.loc[:, sample_id].values, axis=1)
                logging.debug("\nSample {} Locations:  {}\n{}".format(sample_id, sample_locations.shape, sample_locations[:3]))
                matched_locations = (hourly_patient_locations.values == sample_locations)
                logging.debug("\nMatched Location Indicators:  {}\n{}".format(matched_locations.shape, matched_locations[:3]))
                associated_patient_mask = np.any(matched_locations, axis=0)
                logging.debug("\nMatched Location at any Time:  {}\n{}".format(associated_patient_mask.shape, associated_patient_mask[:10]))
                
                # Mask out which patients simultaneously occupy a unit with the current sample at any point in time
#                 associated_patient_mask = (hourly_patient_locations == hourly_patient_locations[sample_id]).any(axis='index')
                associations.loc[sample_id, all_sample_ids[associated_patient_mask]] = True
                ts.update()
                
        logging.debug("Patient associations: {}\n{}".format(associations.shape, associations[:3]))
        association_counts = associations.sum(axis=1)
        logging.debug("Average of {}+/={:.2f} associations per sample".format(association_counts.mean(), association_counts.std()))
        return associations
        
    def get_associated_patients(self, sample_id, associations):
        """ Get the sample IDs for all patients associated with sample_id
            Uses the prebuilt associations matrix
            
        Args:
            sample_id:  (string)
            
        Returns:
            associated_patients:  (list) sample IDs associated with input sample ID
        """
        all_sample_ids = associations.columns
        associated_mask = associations.loc[sample_id, :].values
        logging.debug("Mask:  {}\n{}".format(associated_mask.shape, associated_mask[:10]))
        associated_patients = all_sample_ids[associated_mask]
        logging.debug("Patients:  {}\n{}".format(associated_patients.shape, associated_patients[:10]))
        associated_patients = [sample_id] + [p for p in associated_patients if p != sample_id]
#         associated_indices = np.where(associations[sample_id, :].values == 1)[0]
#         associated_patients = [sample_id] + [all_sample_ids[ai] for ai in associated_indices
#                                              if all_sample_ids[ai] != sample_id]
        logging.debug("Re-ordered Patients:  {}\n{}".format(len(associated_patients), associated_patients[:10]))
        return associated_patients  # first sample ID in list is always the input sample ID
    
    def __len__(self):
        return len(self.sample_ids)
    
    def __getitem__(self, index):
        """ Retrieve a single sample and bundle the data for subsequent batching (performed by DataLoader)
        """
        # Get encounter ID (CSN)
        sample_id = self.sample_ids[index]
        logging.debug("Retrieving sample:  {}".format(sample_id))
        
        # Get all associated patients
        # Defined as anyone in a unit with a batch patient simultaneously, or within params.association_history hours prior
        associated_patients = self.get_associated_patients(sample_id, self.patient_associations)
        logging.debug("Found {} associated patients".format(len(associated_patients)))
        
        # Create mask/index for positions of batch patient(s) within batch-associated patients
        # No longer necessary if duplicating batch-associated data?
        sample_index = 0
        
        # Get all static data (cohort-level, e.g. non-hourly)
        sample_static_features = self.cohort_df.loc[associated_patients, :].values
        logging.debug("Associated patients' static features: {}".format(sample_static_features.shape))
        
        # Bundle all hourly data for associated patients
        # Initialize sample arrays
        logging.debug("Reformatting hourly features...")
        sample_hourly_features = np.full(fill_value=self.params.pad_value,
                                          shape=(len(associated_patients), self.params.pad_length, len(self.params.hourly_features)),
                                          dtype=np.float32)
        sample_hourly_lengths = np.zeros(len(associated_patients), dtype=np.int32)
        
        # Fill sample arrays:  Pad and mask time dimension to a uniform shape
        for asi, associated_sample_id in enumerate(associated_patients):
            associated_hourly_df = self.hourly_dataframes[associated_sample_id].loc[:, self.hourly_features]
            logging.debug("Associated sample {} hourly DF:  {}".format(associated_sample_id, associated_hourly_df.shape))
            sample_hourly_lengths[asi] = min(self.params.pad_length, len(associated_hourly_df))
            logging.debug("Sample pad length: {}".format(sample_hourly_lengths[asi]))
            clipped_df = associated_hourly_df.iloc[:sample_hourly_lengths[asi], :]
            logging.debug("Hourly data clipped to pad length:  {}\n{}".format(clipped_df.shape, clipped_df[:3]))
            sample_hourly_features[asi, :sample_hourly_lengths[asi], :] = clipped_df
            logging.debug("All data clipped and padded to pad length:  {}\n{}".format(sample_hourly_features.shape, sample_hourly_features[:3]))
        logging.debug("Associated patients' hourly features: {}\n{}".format(sample_hourly_features.shape, sample_hourly_features[:3]))
                
        # Create mask for associated patient/unit location matching
        # Allows scattering the short-term hourly data on a per-sample timescale to the global time indices
        # Drop head and tail of all NaN (leftover global time indices for non-associated patients)
        logging.debug("Reformatting hourly locations")
        sample_hourly_locations = self.hourly_patient_locations.loc[:, associated_patients]
        first_idx = sample_hourly_locations.first_valid_index()
        last_idx = sample_hourly_locations.last_valid_index()
        sample_hourly_locations = sample_hourly_locations.loc[first_idx:last_idx].values
        logging.debug("Hourly locations f:  {}\n{}".format(sample_hourly_locations.shape, sample_hourly_locations[:3]))
        
        # Bundle all arrays/values for batching, and return
        sample = (sample_static_features, sample_hourly_locations, sample_hourly_features, sample_hourly_lengths)
        return sample
    
# for reference, from forward()
# Extract bundled batch data
#         batch_samples, patient_inputs = inputs
#         patient_static_features, patient_locations, patient_hourly_features, patient_hourly_lengths = patient_inputs
        
test_sample_ids = pd.Series([330000197395, 330000372959, 330000804075, 330000699855, 330000920644, 330000322138, 330000881688, 330000563672, 330000535940, 330000503459])
test_dataset = ITANStrainDataset(test_sample_ids, params)
test_batch = next(iter(test_dataset))
logging.info("Dataset created!")

# # Construct Skorch datasets for training
# logging.info("Constructing Strain datasets for each cross-validation split.")
# datasets = {}
# for split in ["train", "val"]:
#     logging.info("Building dataset for {} split".format(split))
#     datasets[split] = ITANStrainDataset(cv_split_samples[split], params)
#     logging.info("{} split:  {} samples:\n{}\nX: {}\nY: {}".format(split, len(cv_split_samples[split]), cv_split_samples[split][:3], None, datasets[split].Y.shape))
    
#     logging.info("y values:  {}:  mean {:.2f} +/- {:.2f}".format(
#         datasets[split].Y.shape,
#         np.mean(datasets[split].Y),
#         np.std(datasets[split].Y)))  
    
#     # Test iteration through dataset
#     if False:
#         with tqdm(total=len(datasets[split]), desc="Testing dataset iterator") as st:
#             for sample in datasets[split]:
#                 st.update()
# logging.info("\nDatasets defined!")

NameError: name 'sample_hourly_locations' is not defined

In [ ]:
logging.info(len(test_dataset))
# logging.info(test_dataset)

In [ ]:
logging.info(list(cv_split_samples[split][:10]))

In [24]:
batch_static_features, batch_hourly_locations, batch_hourly_features, batch_hourly_lengths = test_batch
logging.info("Batched static features:   {}\n{}".format(batch_static_features.shape, batch_static_features))
logging.info("\nBatched hourly locations:  {}\n{}".format(batch_hourly_locations.shape, batch_hourly_locations))
logging.info("\nBatched hourly features:   {}\n{}".format(batch_hourly_features.shape, batch_hourly_features))
logging.info("\nBatched hourly lengths:    {}\n{}".format(batch_hourly_lengths.shape, batch_hourly_lengths))

Batched static features:   (1, 5)
[[ 0.06796826 -0.8852548   0.6510309  -1.          1.        ]]

Batched hourly locations:  (81, 1)
ENCOUNTER_ID        330000197395
2018-06-13 05:00:00         WARD
2018-06-13 06:00:00         WARD
2018-06-13 07:00:00         WARD
2018-06-13 08:00:00         WARD
2018-06-13 09:00:00         WARD
2018-06-13 10:00:00         WARD
2018-06-13 11:00:00         WARD
2018-06-13 12:00:00         WARD
2018-06-13 13:00:00         WARD
2018-06-13 14:00:00         WARD
2018-06-13 15:00:00         WARD
2018-06-13 16:00:00         WARD
2018-06-13 17:00:00         WARD
2018-06-13 18:00:00         WARD
2018-06-13 19:00:00         WARD
2018-06-13 20:00:00         WARD
2018-06-13 21:00:00         WARD
2018-06-13 22:00:00         WARD
2018-06-13 23:00:00         WARD
2018-06-14 00:00:00         WARD
2018-06-14 01:00:00         WARD
2018-06-14 02:00:00         WARD
2018-06-14 03:00:00         WARD
2018-06-14 04:00:00         WARD
2018-06-14 05:00:00         WARD
2018-06-

# Data Loader

1.  Define dataset (single facility per epoch)
2.  Load all static data to CPU memory
3.  Load all hourly data to CPU memory
4.  Randomly select batch of B patients
5.  Find all associated patients for the batch
        a. Look at full location (unit) history for each batch patient
        b. Find all other patients who were in same unit as the batch patient at any time
        c. (future) All patients in same unit less than N hours prior to batch patient
6.  Reformat all batch data to a common time scale
7.  Create location masks for NN grouping
8.  Push batch data to GPU and Tensor format for PyTorch


## To Do
8.  Feed batch data through Neural Network and compute loss
9.  Train and evaluate on full dataset